In [7]:
from Tools import *
from U_net import *

In [8]:
crop_size = (512, 512)
# crop_size = (420, 600)
train = GunsDataset(crop_size, 'train')
val = GunsDataset(crop_size, 'valid')
test = GunsDataset(crop_size, 'test')

read 624 examples
read 59 examples
read 30 examples


In [9]:
batch_size = 8
train_iter = torch.utils.data.DataLoader(train, batch_size, shuffle=True, num_workers=1)
val_iter = torch.utils.data.DataLoader(val, batch_size, shuffle=False, num_workers=1)
test_iter = torch.utils.data.DataLoader(test, batch_size, shuffle=False, num_workers=1)

In [10]:
for X, Y in train_iter:
    print(X.shape)
    print(Y.shape)
    break

torch.Size([8, 3, 512, 512])
torch.Size([8, 512, 512])


In [11]:
net = create_unet()

C:\Users\roirt\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           9,408
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          36,864
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
       BasicBlock-11         [-1, 64, 128, 128]               0
           Conv2d-12         [-1, 64, 128, 128]          36,864
      BatchNorm2d-13         [-1, 64, 128, 128]             128
             ReLU-14         [-1, 64, 1

In [12]:
def loss(inputs, targets):
    return F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)
    
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

def binnary_IoU(y_hat, y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    intersection = float((y_hat.type(y.dtype) * y).sum())
    union = float((y_hat.type(y.dtype) + y).clamp(0, 1).sum())
    return intersection, union

In [13]:
num_epochs, lr, wd, device = 100, 0.005, 1e-3, torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trainer = torch.optim.SGD(net.parameters(), lr=lr)#, weight_decay=wd)
model = net.to(device)

for epoch in range(num_epochs):
    L = 0.0
    N = 0
    Acc = 0.0
    Acc_N = 0
    ValAcc = 0.0
    ValN = 0.0
    Union = 0.0
    Intersection = 0.0
    ValUnion = 0.0
    ValIntersection = 0.0
    for X, y in train_iter:
        X, y = X.to(device), y.to(device)
        l = loss(model(X),y)
        trainer.zero_grad()
        l.mean().backward()
        trainer.step()
        L += l.sum()
        N += l.numel()
        Acc += accuracy(model(X), y)
        IoU = binnary_IoU(model(X), y)
        Intersection += IoU[0]
        Union += IoU[1]
        Acc_N += y.numel()
    for X, y in val_iter:
        X, y = X.to(device), y.to(device)
        ValN += y.numel()
        ValAcc += accuracy(model(X), y)
        IoU = binnary_IoU(model(X), y)
        ValIntersection += IoU[0]
        ValUnion += IoU[1]
    print(f'epoch {epoch + 1}/{num_epochs}, loss {(L/N):f}\
          , Train accuracy  {(Acc/Acc_N):f}, Train IoU  {(Intersection/Union):f}\
          , Val accuracy {(ValAcc/ValN):f}, Val IoU {(ValIntersection/ValUnion):f}')

epoch 1/100, loss 0.394399          , Train accuracy  0.865605, Train IoU  0.058251          , Val accuracy 0.898950, Val IoU 0.057716
epoch 2/100, loss 0.340826          , Train accuracy  0.883451, Train IoU  0.075381          , Val accuracy 0.911727, Val IoU 0.091471
epoch 3/100, loss 0.304974          , Train accuracy  0.893580, Train IoU  0.102780          , Val accuracy 0.918060, Val IoU 0.128115
epoch 4/100, loss 0.281675          , Train accuracy  0.897785, Train IoU  0.148351          , Val accuracy 0.921419, Val IoU 0.172991
epoch 5/100, loss 0.264479          , Train accuracy  0.902973, Train IoU  0.176777          , Val accuracy 0.924092, Val IoU 0.219631
epoch 6/100, loss 0.248575          , Train accuracy  0.907369, Train IoU  0.216167          , Val accuracy 0.926919, Val IoU 0.249538
epoch 7/100, loss 0.240564          , Train accuracy  0.911283, Train IoU  0.243287          , Val accuracy 0.931215, Val IoU 0.255430
epoch 8/100, loss 0.228552          , Train accuracy  0

In [14]:
Union=0.0
Intersection=0.0
for X, y in test_iter:
    X, y = X.to(device), y.to(device)
    IoU = binnary_IoU(model(X), y)
    Intersection += IoU[0]
    Union += IoU[1]
print(Intersection/Union)
# next_batch = next(iter(test_iter))
# X, y = next_batch

0.5011350482870618


In [15]:
torch.save(model.to('cpu').state_dict(), 'modelo.pt')

In [ ]:
j=6
a=X[j].unsqueeze(0)
model=model.to('cpu')
pred = model(a).argmax(dim=1)

In [ ]:
plt.imshow(pred.permute(1, 2, 0).numpy())
plt.axis('off')  # Opcional: desactiva los ejes
plt.show()

In [ ]:
plt.imshow((y[j].unsqueeze(0).to('cpu')).permute(1, 2, 0).numpy())
plt.axis('off')  # Opcional: desactiva los ejes
plt.show()